In [ ]:
import requests
import io

test_url = 'https://api.dev-rcsed2.voxastro.org/spec/list/download/?q=cone(0.800578,%20-6.091818,%200.2)&resolver=param&fmt=csv'
with requests.get(test_url, stream=True) as urlo:
    # This opens the data as using the astropy fits interface (using io.BytesIO() to stream it into memory
    #  first so that fits.open can access it as an already opened file handler).
    # with fits.open(io.BytesIO(urlo.content)) as full_fits:
    with io.BytesIO(urlo.content) as rcsed_search_cont:
        rcsed_res = pd.read_csv(rcsed_search_cont)
        